In [12]:
from braket.aws import AwsDevice
from braket.aws import AwsQuantumJob, AwsSession
from braket.jobs.config import InstanceConfig

import boto3

import os
import json
import time
import altair as alt
import pandas as pd
import numpy as np

In [13]:
import copy

c = copy.deepcopy

def obtain_default_bucket(target: str) -> str:
    with open(os.path.abspath('../../..') + '/.default-setting') as f:
        lines = f.readlines()
        for line in lines:
            if (line.startswith(target+'=')):
                return line.split('=')[1].strip()
            
def get_key(single_dict):
    for k in single_dict.keys():
        return k

def parse_params(params_list, hp, hp_list):
    params = params_list[0]
    k = get_key(params)
    ps = params[k]
    for p in ps:
        hp[k] = p
        if len(params_list) == 1:
            hp_list.append(c(hp))
        else:
            parse_params(params_list[1:], hp, hp_list)

def get_quantum_device(device_name):
    device_arn = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"
    try:
        device = AwsDevice.get_devices(names=[device_name])
        device_arn = device[0].arn
    except Exception as e:
        print(f"fail to get {device_name}: {e}, use sv1 instead")
    return device_arn

# def upload_data(dir_name, suffix_check, aws_session=AwsSession()):

#     stream_s3_uri = aws_session.construct_s3_uri(obtain_default_bucket("bucketName"), dir_name)
#     return_path = None
    
#     def _check_upload(file_name, check_list):
#         file_end = file_name.split('.')[-1]
#         print(f"file end is {file_end}")
#         if file_end in check_list:
#             path = f"{stream_s3_uri}/" + file_name.split('/')[-1]
#             aws_session.upload_to_s3(file_name, path)
            
#     if os.path.isdir(dir_name):
#         dir_list = os.listdir(dir_name)
#         print(f"!! dir_list {dir_list}")
#         for file_name in dir_list:
#             _check_upload(os.path.join(dir_name,file_name), suffix_check)
#         return_path = stream_s3_uri
#     else:
#         _check_upload(file_name, suffix_check)
#         single_file_name = file_name.split('/')[-1]
#         return_path = f"{stream_s3_uri}/{single_file_name}"
def upload_data(dir_name):
    
    bucket_name = obtain_default_bucket("bucketName")
    
    # create an S3 client
    s3 = boto3.client('s3')

    # sync the local folder to the S3 bucket
    for root, dirs, files in os.walk(dir_name):
        for file in files:
            local_path = os.path.join(root, file)
            s3.upload_file(local_path, bucket_name, local_path)

    return_path = f"s3://{bucket_name}/{dir_name}"
        
    return return_path



def queue_check(jobs):
    # check current running or queued jobs
    region = boto3.client('s3').meta.region_name
    braket = boto3.client("braket", region_name=region)
    p = braket.get_paginator("search_jobs")
    paginator = p.paginate(filters=[]).build_full_result()
    
    existing_job_count = 0
    if "jobs" in paginator:
        for job in paginator["jobs"]:
            if job["status"] in ["RUNNING", "QUEUED"]:
                existing_job_count = existing_job_count + 1
    
    check_pass = True
    if existing_job_count >= 4:
        check_pass = False
    
    print(f"There are {existing_job_count} jobs in RUNNING or QUEUED status")
    
    return check_pass

def get_result(result, target, dm):
    return [dm, result["time"], target]

def get_dm(target):
    file_name = './rna-folding-data/' + target + '.fasta.txt'
    str_len = -100
    with open(file_name) as file:
        fasta_lines = file.readlines()
        str_len = len(fasta_lines[1])
    return str_len

def display_results(results, experiments_params):
    sorted_results = {}
    
    for device in experiments_params["params"][4]["device"]:
        sorted_results[str(device)] = []
        # for target in results[0].keys():
        #     sorted_results[str(device)][target] = []
    
    max_offset = 10
    for result in results:
        for target in result.keys():
            device = result[target]["hypermeter"]["device"]
            
            dm = get_dm(target)

            if len(sorted_results[device]) == 0:
                sorted_results[device].append(get_result(result[target],target,dm))
                continue

            last_idx = 0
            for idx, sorted_result in enumerate(sorted_results[device]):
                sorted_dm = float(sorted_result[0])

                last_sorted_dm = float(sorted_results[device][last_idx][0])

                if last_sorted_dm < dm and dm < sorted_dm:
                    sorted_results[device].insert(idx,get_result(result[target],target,dm))
                    break
                elif dm < last_sorted_dm and idx == 0:
                    sorted_results[device].insert(last_idx,get_result(result[target],target,dm))
                    break
                elif dm > sorted_dm and idx == len(sorted_results[device])-1:
                    sorted_results[device].insert(idx+1,get_result(result[target],target,dm))

                last_idx = idx
    # print(f"sorted result {sorted_results}")
    return sorted_results

# Step 1: Prepare parameters for batch evaluation

In this part, we set the parameters for batch evaluation

In [14]:
# parameters for experiments
experiment_name = "QfoldHybridJobs"
data_path = "protein-folding-data"
suffix_check = ["json"]
experiments_params =  {
    "version": "1",
    "params": [
        {"method": ["qfold-cc", "qfold-qc"]},
        {"initialization": ["minifold", "random"]},
        {"shots": [10000]},
        {"mode": ["local-simulator"]},
        {"device": [{"qc": "null", "cc": "ml.m5.large"},{"qc": "null", "cc": "ml.m5.4xlarge"}]}
    ]
}

hp = {}
hybrid_job_params = []
parse_params(experiments_params['params'], hp, hybrid_job_params)

print(f"parameters for experiments: \n {hybrid_job_params}")

parameters for experiments: 
 [{'method': 'qfold-cc', 'initialization': 'minifold', 'shots': 10000, 'mode': 'local-simulator', 'device': {'qc': 'null', 'cc': 'ml.m5.large'}}, {'method': 'qfold-cc', 'initialization': 'minifold', 'shots': 10000, 'mode': 'local-simulator', 'device': {'qc': 'null', 'cc': 'ml.m5.4xlarge'}}, {'method': 'qfold-cc', 'initialization': 'random', 'shots': 10000, 'mode': 'local-simulator', 'device': {'qc': 'null', 'cc': 'ml.m5.large'}}, {'method': 'qfold-cc', 'initialization': 'random', 'shots': 10000, 'mode': 'local-simulator', 'device': {'qc': 'null', 'cc': 'ml.m5.4xlarge'}}, {'method': 'qfold-qc', 'initialization': 'minifold', 'shots': 10000, 'mode': 'local-simulator', 'device': {'qc': 'null', 'cc': 'ml.m5.large'}}, {'method': 'qfold-qc', 'initialization': 'minifold', 'shots': 10000, 'mode': 'local-simulator', 'device': {'qc': 'null', 'cc': 'ml.m5.4xlarge'}}, {'method': 'qfold-qc', 'initialization': 'random', 'shots': 10000, 'mode': 'local-simulator', 'device':

In [15]:
# Upload dataset to S3
s3_path = upload_data(data_path)
print(f"upload data to s3 path: {s3_path}")

upload data to s3 path: s3://amazon-braket-us-east-1-002224604296/protein-folding-data


# Step 2: Prepare image for experiment

In this part, we use the following code to prepare the image for experiment. For the first run, 
please run build_and_push.sh to create the image. For future experiments, avoid running
build_and_push.sh unless you want to rebuild the image

In [16]:
!cp /home/ubuntu/psi4conda/bin/psi4 QfoldHybridJobs/psi4

In [17]:
account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.client('s3').meta.region_name
image_name = f"amazon-braket-{experiment_name.lower()}-jobs"
image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{image_name}:latest"

print(f"the hybrid job image for {account_id} in region {region}: {image_uri}")

# For the first run, please use the following code to create the image for this application. For future experiments, comment
# the following code unless you want to rebuild the image
# !sh build_and_push.sh {image_name}

the hybrid job image for 002224604296 in region us-east-1: 002224604296.dkr.ecr.us-east-1.amazonaws.com/amazon-braket-qfoldhybridjobs-jobs:latest


In [18]:
hybrid_jobs_json = f"{experiment_name}-hybrid-jobs.json"
print(f"job info will be saved in {hybrid_jobs_json}")

job info will be saved in QfoldHybridJobs-hybrid-jobs.json


# Step 3: Launch Amazon Braket Hybrid Jobs for experiment

In this part, we use the following code to launch the same number of hybrid jobs as the sets of parameters for this experiments.
When the number of jobs exceeds 5 RPS, this thread will wait. The default setting of this experiment will take around **7 hours** to 
finish.

In [21]:
# Long runnning cell due to Burst rate of CreateJob requests < 5 RPS
# sudo apt-get install python-prctl at first
# https://stackoverflow.com/questions/34361035/python-thread-name-doesnt-show-up-on-ps-or-htop
from threading import Thread
import threading
import setproctitle

def launch_hybrid_jobs(hybrid_job_params=hybrid_job_params, hybrid_jobs_json=hybrid_jobs_json):
    setproctitle.setproctitle(threading.current_thread().name)
    # parse evaluation parameters and trigger hybrid jobs:
    jobs = []
    names = []

    job_name = experiment_name.lower()
    device_param_list = ["shots", "device"]

    for job_param in hybrid_job_params:
        
        algorithm_param_name = ""
        for k,v in job_param.items():
            if k not in device_param_list:
                algorithm_param_name = algorithm_param_name+f"-{v[0]}"
        algorithm_param_name=algorithm_param_name[1:]
        quantum_device = get_quantum_device(job_param['device']['qc'])
        classical_device = job_param['device']['cc']

        device_name = classical_device.replace(".","-")
        device_name = device_name.replace("x","")
        
        name = f"{algorithm_param_name}-{device_name}-" + str(int(time.time()))
        name = name.lower()
        # name = f"{experiment_name}-"+ str(int(time.time()))
        print(f"name is {name}")

        tmp_job = AwsQuantumJob.create(
            device=quantum_device,
            source_module=f"{experiment_name}",
            entry_point=f"{experiment_name}.{job_name}:main",
            job_name=name,
            hyperparameters=job_param,
            input_data=s3_path,
            instance_config=InstanceConfig(instanceType=classical_device),
            image_uri=image_uri,
            wait_until_complete=False,
        )
        
#         from braket.jobs.local import LocalQuantumJob
        
#         tmp_job = LocalQuantumJob.create(
#             device=quantum_device,
#             source_module=f"{experiment_name}",
#             entry_point=f"{experiment_name}.{job_name}:main",
#             hyperparameters=job_param,
#             input_data=s3_path,
#             image_uri=image_uri,
#         )   
        
        print(f"Finish create {experiment_name} with {name}")

        jobs.append(tmp_job)
        names.append(name)


        while not queue_check(jobs):
            time.sleep(5)
    jobs_arn = []

    for job in jobs:
        jobs_arn.append(job.arn)

    jobs_states = {
        "experiment_name": experiment_name,
        "hybrid-jobs-arn": jobs_arn,
        "names": names
    }
    
    
    # save hybrid job arn for further analysis
    json_object = json.dumps(jobs_states, indent=4)

    with open(hybrid_jobs_json, "w") as outfile:
        outfile.write(json_object)
        
    print(f"Finish launch all the hybrid jobs and save all the files")

# remove existing hybrid_jobs_json file
!rm {hybrid_jobs_json}

t = Thread(target=launch_hybrid_jobs, name="launch-hybrid-job", daemon=True).start()

# launch_hybrid_jobs()

rm: cannot remove 'QfoldHybridJobs-hybrid-jobs.json': No such file or directory
fail to get null: list index out of range, use sv1 instead
name is q-m-l-ml-m5-large-1679454022
Finish create QfoldHybridJobs with q-m-l-ml-m5-large-1679454022
There are 1 jobs in RUNNING or QUEUED status
fail to get null: list index out of range, use sv1 instead
name is q-m-l-ml-m5-4large-1679454037
Finish create QfoldHybridJobs with q-m-l-ml-m5-4large-1679454037
There are 2 jobs in RUNNING or QUEUED status
fail to get null: list index out of range, use sv1 instead
name is q-r-l-ml-m5-large-1679454044
Finish create QfoldHybridJobs with q-r-l-ml-m5-large-1679454044
There are 3 jobs in RUNNING or QUEUED status
fail to get null: list index out of range, use sv1 instead
name is q-r-l-ml-m5-4large-1679454050
Finish create QfoldHybridJobs with q-r-l-ml-m5-4large-1679454050
There are 4 jobs in RUNNING or QUEUED status
There are 4 jobs in RUNNING or QUEUED status
There are 4 jobs in RUNNING or QUEUED status
There 

In [22]:
# run the following scripts to check the created threads
!ps -aux | grep launch-hybrid-job

ubuntu     52285  0.7  1.0 2466724 343908 ?      Ssl  02:42   0:16 launch-hybrid-job
ubuntu     53803  0.0  0.0   8756  3432 pts/21   Ss+  03:20   0:00 /bin/bash -c ps -aux | grep launch-hybrid-job
ubuntu     53808  0.0  0.0   8176   728 pts/21   S+   03:20   0:00 grep launch-hybrid-job


# Step 4: Jobs finish and visualize results

Please use the following code to check the status of hybrid jobs. The status of hybrid jobs can also be checked in the Amazon Braket console. Optionally, if the email if input when deploying the solution, emails will be sent at the same number of hybrid jobs once 
the status of jobs changes.

In [23]:
# run the following code to test whether all the jobs finish
results = []
if os.path.exists(hybrid_jobs_json):
    # recover hybrid jobs and show result
    jobs_states_load = None
    with open(hybrid_jobs_json, "r") as outfile:
        jobs_states_load = json.load(outfile)

    completed_jobs_arn = set()

    for job_name, job_arn in zip(jobs_states_load["names"], jobs_states_load["hybrid-jobs-arn"]):
        current_job = AwsQuantumJob(job_arn)
        print(f"the state of job {job_name} is : {current_job.state()}")
        if current_job.state() == 'COMPLETED':
            completed_jobs_arn.update({job_arn})

    whole_jobs_num = len(jobs_states_load["names"])

    if len(completed_jobs_arn) == whole_jobs_num:
        print(f"all jobs completed")
        for job_arn in completed_jobs_arn:
            current_job = AwsQuantumJob(job_arn)
            results.append(current_job.result())
        # display results
        results = display_results(results, experiments_params)
else:
    print(f"JSON file for job arns not generated! please wait for the thread(launch-hybrid-job) to finish")

the state of job q-m-l-ml-m5-large-1679454022 is : COMPLETED
the state of job q-m-l-ml-m5-4large-1679454037 is : COMPLETED
the state of job q-r-l-ml-m5-large-1679454044 is : COMPLETED
the state of job q-r-l-ml-m5-4large-1679454050 is : COMPLETED
the state of job q-m-l-ml-m5-large-1679454236 is : COMPLETED
the state of job q-m-l-ml-m5-4large-1679454268 is : COMPLETED
the state of job q-r-l-ml-m5-large-1679454314 is : COMPLETED
the state of job q-r-l-ml-m5-4large-1679454322 is : COMPLETED
all jobs completed


KeyError: 'hypermeter'

In [11]:
rename_result = {}
device_list = []
x_list = []
y_list = []
for k,vs in results.items():
    k = k.replace("\'","\"")
    dict_k = json.loads(k)
    device_name = None
    if dict_k['qc'] == 'null':
        device_name = dict_k['cc']
    else:
        device_name = dict_k['qc']
    for v in vs:
        device_list.append(device_name)
        x_list.append(v[0])
        y_list.append(v[1])
source = pd.DataFrame({
    "Sequence Length": np.array(x_list),
    "Time to Solution": np.array(y_list),
    "Device": np.array(device_list),
})

alt.Chart(source).mark_line(point = True).encode(
    x='Sequence Length',
    y='Time to Solution',
    color='Device',
).properties(
    title = f"{experiment_name} experiments",
    width = 700,
    height = 600,
).interactive()

AttributeError: 'list' object has no attribute 'items'

fail to get null: list index out of range, use sv1 instead
name is qfold-cc-minifold-local-simulator-ml-m5-large-1679452955


Exception in thread launch-hybrid-job:
Traceback (most recent call last):
  File "/opt/conda/envs/qfold-python3.7/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/qfold-python3.7/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-13347a113ad7>", line 44, in launch_hybrid_jobs
    wait_until_complete=False,
  File "/opt/conda/envs/qfold-python3.7/lib/python3.7/site-packages/braket/aws/aws_quantum_job.py", line 198, in create
    job_arn = aws_session.create_job(**create_job_kwargs)
  File "/opt/conda/envs/qfold-python3.7/lib/python3.7/site-packages/braket/aws/aws_session.py", line 244, in create_job
    response = self.braket_client.create_job(**boto3_kwargs)
  File "/opt/conda/envs/qfold-python3.7/lib/python3.7/site-packages/botocore/client.py", line 508, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/opt/conda/envs/qfold-python3.7/lib/python3